In [1]:
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [37]:
def weekly_target_percentages(targets, column):
    try:
        total_targets = df[column].sum()
        value = round(targets / total_targets, 4)
    except:
        value = targets
    return value

In [146]:
teams = ['rav', 'buf', 'cin', 'cle', 'den', 'htx', 'clt', 'jax', 'kan', 'sdg', 'rai', 'mia', 'nwe', 'nyj', 'pit', 'oti',
        'crd', 'atl', 'car', 'chi', 'dal', 'det', 'gnb', 'ram', 'min', 'nor', 'nyg', 'phi', 'sea', 'sfo', 'tam', 'was']
years = [2013, 2014, 2015, 2016, 2017, 2018]
weeks = [str(x) for x in range(1,18)] #17 weeks in a regular season
df_list = []
for team in teams:
    for year in years:
        r = re.get(f'https://footballguys.com/subscribers/teams/teampage-{team}-2.php')
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.select_one(f'h2:-soup-contains("{str(year)}") + table')
        df = pd.read_html(str(table))[0]
        
        df = df[~df['Player'].isin(['WR TOTAL', 'TE TOTAL', 'RB TOTAL'])]   #we dont want these uncessary rows
        df['Player'] = df['Player'].map(lambda x: x + ' ' + str(year + 1)) #match player format with main df
        df['tgt_per_game'] = round(df[weeks].sum(axis=1) / df[weeks].count(axis=1), 4)
        for col in weeks:
            df[col] = df[col].map(lambda x: weekly_target_percentages(x, col))
        df['tgt_pct_per_game'] = round(df[weeks].sum(axis=1) / df[weeks].count(axis=1), 4)
        df = df[['Player', 'tgt_per_game', 'tgt_pct_per_game']]
        
        df_list.append(df)
df = pd.concat(df_list)
df = df.groupby('Player').mean()
df.reset_index(inplace=True)

In [147]:
df.to_csv('player_target_per_game_stats.csv', index=False)

In [148]:
df = pd.read_csv('player_target_per_game_stats.csv')
df.head()

,Player,tgt_per_game,tgt_pct_per_game
0,A.J. Derby 2017,1.66665,0.04330
1,A.J. Derby 2018,3.97220,0.10895
2,A.J. Derby 2019,1.25000,0.04610
3,A.J. Green 2014,11.12500,0.30150
4,A.J. Green 2015,8.92310,0.29290


In [114]:
# # used for testing purposes so I didnt query the website too much
# team = 'cle'
# year = 2013
# weeks = [str(x) for x in range(1,18)] #17 weeks in a regular season
# df_list = []
# r = re.get(f'https://footballguys.com/subscribers/teams/teampage-{team}-2.php')
# soup = BeautifulSoup(r.content, 'html.parser')
# table = soup.select_one(f'h2:-soup-contains("{str(year)}") + table')
# df = pd.read_html(str(table))[0]
# df = df[~df['Player'].isin(['WR TOTAL', 'TE TOTAL', 'RB TOTAL'])]
# df['Player'] = df['Player'].map(lambda x: x + ' ' + str(year + 1)) #match player format with main df
# df['tgt_per_game'] = round(df[weeks].sum(axis=1) / df[weeks].count(axis=1), 4)
# for col in weeks:
#     df[col] = df[col].map(lambda x: weekly_target_percentages(x, col))
# df['tgt_pct_per_game'] = round(df[weeks].sum(axis=1) / df[weeks].count(axis=1), 4)
# df = df[['Player', 'tgt_per_game', 'tgt_pct_per_game']]
# df